In [1]:
import qiskit_nature
from qiskit_nature.second_q.drivers import Psi4Driver
import numpy as np
from qiskit_nature.units import DistanceUnit
from qiskit_algorithms import VQE
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
import pylab
from matplotlib import pyplot as plt
import psi4
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature import settings
settings.use_ipython = False

In [2]:
driver_options = {
    'basis': 'sto-3g',
    'scf_type': 'pk', 
    'reference': 'rhf'
}
distances = np.arange(0.5,4.25,0.25)
energies = {}
service = QiskitRuntimeService(channel="ibm_quantum", token="82f6eeb5a67e68bc27f19c170479d281103dddfc005292906679d90a8971f33b84ce8c51ce991c90181edd9734ecc5e9519afc001f203c9447e8247923124966")
backend = service.least_busy(operational=True, simulator=False)

In [3]:
for i, d in enumerate(distances):
    print(f"Step {i}: Calculating energy for distance {d} Angstrom")

    molecule_geometry = f"""
0 1
Li 0.0 0.0 0.0
H  0.0 0.0 {d}
    """


    molecule_geometry = molecule_geometry.strip()


    print(f"Molecule Geometry for distance {d} Angstrom:")
    print(molecule_geometry)


    psi4_input = f"""
molecule {{
{molecule_geometry}
}}

set basis = sto-3g

energy('scf', return_wfn=True)
    """

    try:
        driver = Psi4Driver(psi4_input)
        problem = driver.run()
        print(f"Problem successfully created for distance {d}")
    except Exception as e:
        print(f"Error running Psi4Driver for distance {d}: {e}")
        continue
    
    num_spatial_orbitals = problem.num_spatial_orbitals
    num_particles = problem.num_particles


    print(f"Number of spatial orbitals: {num_spatial_orbitals}")
    print(f"Number of particles: {num_particles}")


    mapper = JordanWignerMapper()
 

    hf_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)


    ansatz = UCCSD(num_spatial_orbitals=num_spatial_orbitals, num_particles =num_particles, initial_state=hf_state)

    vqe = VQE(backend, ansatz, SLSQP())


    calc = GroundStateEigensolver(mapper, vqe)

    # Solve the problem to get the ground state energy
    res = calc.solve(problem)

    # Store the energy result
    energies.append(res.groundenergy)

Step 0: Calculating energy for distance 0.5 Angstrom
Molecule Geometry for distance 0.5 Angstrom:
0 1
Li 0.0 0.0 0.0
H  0.0 0.0 0.5
Problem successfully created for distance 0.5
Number of spatial orbitals: 6
Number of particles: (2, 2)


AlgorithmError: 'The number of qubits of the ansatz does not match the operator, and the ansatz does not allow setting the number of qubits using `num_qubits`.'

In [ ]:
print(energies)